In [76]:
import glob
import gensim
import pandas as pd
import numpy as np
import logging
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

wordnet_lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')
STOPWORDS = set(stopwords.words('english'))
STOPWORDS.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [77]:
D_raw = [[u'layoff', 'cut', 'new', u'panel']]
VECTOR_LEN = len(D_raw[0])

D = gensim.corpora.Dictionary(D_raw)
D.items()

2017-07-28 15:57:31,711 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-07-28 15:57:31,712 : INFO : built Dictionary(4 unique tokens: [u'new', u'cut', u'layoff', u'panel']) from 1 documents (total 4 corpus positions)


[(0, u'new'), (1, u'cut'), (2, u'layoff'), (3, u'panel')]

In [78]:
data = pd.read_csv("data_clf/health_reuter.csv")
data['label'] = 0
data['full_content'] = data['title'] + ' ' + data['long_text']
data.head()

,url,title,tag,time,short_text,long_text,label,full_content
0,https://www.reuters.com/article/us-usa-congres...,U.S. House panel spotlights use of FDA rules t...,NaN,Jul 27 2017,Republican and Democratic lawmakers on Thursda...,WASHINGTON (Reuters) - Republican and Democrat...,0,U.S. House panel spotlights use of FDA rules t...
1,https://www.reuters.com/article/us-amgen-chole...,Amgen gets fast FDA review for adding heart be...,NaN,Jul 27 2017,Amgen Inc said on Thursday the U.S. Food and D...,(Reuters) - Amgen Inc said on Thursday the U.S...,0,Amgen gets fast FDA review for adding heart be...
2,https://www.reuters.com/article/us-health-tran...,Trans teens worse off on health measures than ...,NaN,Jul 27 2017,(Reuters Health) - Teenagers who identify as t...,(Reuters Health) - Teenagers who identify as t...,0,Trans teens worse off on health measures than ...
3,https://www.reuters.com/article/us-health-obes...,Adult colorectal cancer risk tied to weight as...,NaN,Jul 27 2017,(Reuters Health) - Being overweight or obese a...,(Reuters Health) - Being overweight or obese a...,0,Adult colorectal cancer risk tied to weight as...
4,https://www.reuters.com/article/us-health-pert...,Vaccine lessens severity of whooping cough inf...,NaN,Jul 27 2017,(Reuters Health) - Even though vaccinations do...,(Reuters Health) - Even though vaccinations do...,0,Vaccine lessens severity of whooping cough inf...


In [93]:
def line2vec(doc, dictionary):
    doc = [wordnet_lemmatizer.lemmatize(z) for z in tokenizer.tokenize(str(doc).decode('utf-8').lower()) if z not in STOPWORDS] 
    bow_vector = dictionary.doc2bow(doc)
    
    # Convert to multi-hot vector
    bow_vector = [(v, 1) for v, _ in bow_vector]
    
    return bow_vector 

def doc2vec(data_texts):
    return [line2vec(text, D) for text in data_texts]

def predict(text):
    vector = gensim.matutils.sparse2full(line2vec(text, D), VECTOR_LEN)
    baseline_vector = np.ones(VECTOR_LEN)
    print line2vec(text, D)
    return gensim.matutils.cossim(vector, baseline_vector)

In [94]:
data['predict'] = data['full_content'].apply(predict)

[(3, 1)]


TypeError: cannot convert dictionary update sequence element #0 to a sequence